In [1]:
import torch
import pandas as pd
torch.manual_seed(1)

train = pd.read_csv('../input/2021-ai-w3-p2/train_data.csv')
test = pd.read_csv('../input/2021-ai-w3-p2/test_data.csv')
submit = pd.read_csv('../input/2021-ai-w3-p2/sample_submit.csv')

train = train.drop('ID', axis = 1)
test = test.drop('ID', axis = 1)
train
test

,company,model,year,transmission,mileage,fueltype,tax,mpg,engineSize
0,2,Ka+,2017,1,10150,4,145,57.7,1.2
1,2,Kuga,2013,1,50216,0,145,53.3,2.0
2,3,Q5,2016,1,34189,0,200,47.9,2.0
3,1,I10,2020,1,1900,4,145,56.5,1.0
4,0,3 Series,2014,0,79809,0,20,68.9,2.0
...,...,...,...,...,...,...,...,...,...
16826,2,Grand Tourneo Connect,2016,0,29000,0,125,56.6,1.5
16827,4,Yaris,2019,1,2812,4,145,47.9,1.5
16828,2,Kuga,2017,1,46800,0,150,54.3,2.0
16829,3,A7,2017,3,14562,0,145,52.3,3.0


In [2]:
#실수화
import numpy as np
from sklearn.preprocessing import LabelEncoder

model_label = np.concatenate([train['model'].unique(), test['model'].unique()],0)
model_label =np.unique(model_label)

print(train['model'].unique().shape)
print(model_label.shape)

le = LabelEncoder()
le.fit(model_label)

train['model'] = le.transform(train['model'])
test['model'] = le.transform(test['model'])

train.shape
train

(98,)
(98,)


,company,model,year,transmission,mileage,fueltype,tax,mpg,engineSize,price
0,2,26,2019,1,3764,4,145,34.5,2.3,26299
1,1,34,2015,1,27056,4,20,60.1,1.0,5495
2,3,9,2019,3,3375,4,145,34.5,2.0,32400
3,3,12,2019,0,5743,0,145,47.9,2.0,29990
4,2,25,2015,1,32782,4,0,65.7,1.0,7250
...,...,...,...,...,...,...,...,...,...,...
17080,0,0,2018,0,4666,4,145,32.5,3.0,23499
17081,3,10,2020,3,22601,4,145,39.2,2.0,32481
17082,4,93,2018,1,20312,4,145,47.1,1.5,10950
17083,2,26,2018,1,18695,4,145,58.9,1.0,13295


In [3]:
x_train = train.drop('price', axis = 1)
y_train = train['price']
x_test = test
x_train.shape
test

,company,model,year,transmission,mileage,fueltype,tax,mpg,engineSize
0,2,44,2017,1,10150,4,145,57.7,1.2
1,2,46,2013,1,50216,0,145,53.3,2.0
2,3,60,2016,1,34189,0,200,47.9,2.0
3,1,34,2020,1,1900,4,145,56.5,1.0
4,0,2,2014,0,79809,0,20,68.9,2.0
...,...,...,...,...,...,...,...,...,...
16826,2,32,2016,0,29000,0,125,56.6,1.5
16827,4,93,2019,1,2812,4,145,47.9,1.5
16828,2,46,2017,1,46800,0,150,54.3,2.0
16829,3,13,2017,3,14562,0,145,52.3,3.0


In [4]:
#표준화
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(x_train)

x_train = sc.transform(x_train)
x_test = sc.transform(x_test)

x_test

array([[ 0.03403625,  0.39749034,  0.00842929, ...,  0.40414289,
         0.06669112, -0.78679016],
       [ 0.03403625,  0.4706327 , -1.84384596, ...,  0.40414289,
        -0.16803799,  0.54771561],
       [ 0.80629309,  0.98262927, -0.45463952, ...,  1.24918731,
        -0.45611462,  0.54771561],
       ...,
       [ 0.03403625,  0.4706327 ,  0.00842929, ...,  0.48096511,
        -0.11469046,  0.54771561],
       [ 0.80629309, -0.73621634,  0.00842929, ...,  0.40414289,
        -0.22138551,  2.21584783],
       [ 0.80629309,  0.94605809, -1.38077715, ...,  0.71143177,
        -0.32274581,  0.54771561]])

In [5]:
x_train = torch.FloatTensor(x_train)
x_test = torch.FloatTensor(x_test)
y_train = torch.FloatTensor(y_train)

In [6]:
x_train.shape

layer = torch.nn.Linear(9,1)

model = torch.nn.Sequential(layer)

In [7]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr = 0.001)
loss = torch.nn.MSELoss()


epochs = 10000

for e in range(epochs + 1):
    hypothesis = model(x_train)
    
    cost = loss(hypothesis, y_train.unsqueeze(1))
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if e % 1000 == 0 :
        print("epoch : ", e, "cost : ", cost.item())
    

epoch :  0 cost :  383205536.0
epoch :  1000 cost :  36507592.0
epoch :  2000 cost :  29252196.0
epoch :  3000 cost :  28636250.0
epoch :  4000 cost :  28494310.0
epoch :  5000 cost :  28455772.0
epoch :  6000 cost :  28445046.0
epoch :  7000 cost :  28442016.0
epoch :  8000 cost :  28441142.0
epoch :  9000 cost :  28440886.0
epoch :  10000 cost :  28440808.0


In [8]:
predict = model(x_test)
predict

tensor([[13353.4072],
        [11128.9170],
        [17715.4961],
        ...,
        [17682.7500],
        [33265.4219],
        [10016.1494]], grad_fn=<AddmmBackward>)

In [9]:
submit['price'] = predict.detach().numpy()

submit.to_csv("submission.csv", index = False)

s = pd.read_csv("submission.csv")
s

,ID,price
0,0,13353.407
1,1,11128.917
2,2,17715.496
3,3,16207.815
4,4,8051.345
...,...,...
16826,16826,11203.960
16827,16827,22208.492
16828,16828,17682.750
16829,16829,33265.420
